Step 0. Unzip enron1.zip into the current directory.

Step 1. Traverse the dataset and create a Pandas dataframe. This is already done for you and should run without any errors. You should recognize Pandas from task 1.

In [3]:
import pandas as pd
import os
print(pd.__version__)
def read_spam():
    category = 'spam'
    directory = './enron1/spam'
    return read_category(category, directory)

def read_ham():
    category = 'ham'
    directory = './enron1/ham'
    return read_category(category, directory)

def read_category(category, directory):
    emails = []
    for filename in os.listdir(directory):
        if not filename.endswith(".txt"):
            continue
        with open(os.path.join(directory, filename), 'r') as fp:
            try:
                content = fp.read()
                emails.append({'name': filename, 'content': content, 'category': category})
            except:
                print(f'skipped {filename}')
    return emails

# Reading ham and spam emails
ham = read_ham()
spam = read_spam()

# Creating DataFrames for ham and spam emails
df_ham = pd.DataFrame.from_records(ham)
df_spam = pd.DataFrame.from_records(spam)

# Concatenating ham and spam DataFrames
df = pd.concat([df_ham, df_spam], ignore_index=True)

# Print the first few rows of the DataFrame to confirm it worked
print(df.head())

2.2.3
skipped ._0120.2000-01-04.farmer.ham.txt
skipped ._3081.2000-12-12.farmer.ham.txt
skipped ._3525.2001-02-05.farmer.ham.txt
skipped ._4209.2001-04-05.farmer.ham.txt
skipped ._5100.2001-12-07.farmer.ham.txt
skipped ._0230.2000-01-20.farmer.ham.txt
skipped ._4719.2001-07-02.farmer.ham.txt
skipped ._4465.2001-05-02.farmer.ham.txt
skipped ._2601.2000-10-20.farmer.ham.txt
skipped ._1031.2000-05-03.farmer.ham.txt
skipped ._3424.2001-01-25.farmer.ham.txt
skipped ._1751.2000-07-26.farmer.ham.txt
skipped ._1657.2000-07-17.farmer.ham.txt
skipped ._1836.2000-08-01.farmer.ham.txt
skipped ._0493.2000-02-25.farmer.ham.txt
skipped ._4669.2001-06-14.farmer.ham.txt
skipped ._5082.2001-11-21.farmer.ham.txt
skipped ._2727.2000-10-31.farmer.ham.txt
skipped ._1537.2000-06-29.farmer.ham.txt
skipped ._3200.2000-12-27.farmer.ham.txt
skipped ._3861.2001-03-20.farmer.ham.txt
skipped ._4319.2001-04-19.farmer.ham.txt
skipped ._4381.2001-04-25.farmer.ham.txt
skipped ._4553.2001-05-16.farmer.ham.txt
skipped ._

Step 2. Data cleaning is a critical part of machine learning. You and I can recognize that 'Hello' and 'hello' are the same word but a machine does not know this a priori. Therefore, we can 'help' the machine by conducting such normalization steps for it. Write a function `preprocessor` that takes in a string and replaces all non alphabet characters with a space and then lowercases the result.

In [4]:
import re

def preprocessor(e):
    result = []
    for k in e:
        if k.isalpha():  
            result.append(k.lower()) 
        else:
            result.append(' ')  
    return ''.join(result).strip()

Step 3. We will now train the machine learning model. All the functions that you will need are imported for you. The instructions explain how the work and hint at which functions to use. You will likely need to refer to the scikit learn documentation to see how exactly to invoke the functions. It will be handy to keep that tab open.

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# The CountVectorizer converts a text sample into a vector (think of it as an array of floats).
# Each entry in the vector corresponds to a single word and the value is the number of times the word appeared.
# Instantiate a CountVectorizer. Make sure to include the preprocessor you previously wrote in the constructor.


vecorizer = CountVectorizer(preprocessor=preprocessor)
X = vecorizer.fit_transform(df['content'])


# Use train_test_split to split the dataset into a train dataset and a test dataset.
# The machine learning model learns from the train dataset.
# Then the trained model is tested on the test dataset to see if it actually learned anything.
# If it just memorized for example, then it would have a low accuracy on the test dataset and a high accuracy on the train dataset.

X_train, X_test, y_train, y_test = train_test_split(X, df['category'], test_size=0.2, random_state=42)



# Use the vectorizer to transform the existing dataset into a form in which the model can learn from.
# Remember that simple machine learning models operate on numbers, which the CountVectorizer conveniently helped us do.






# Use the LogisticRegression model to fit to the train dataset.
# You may remember y = mx + b and Linear Regression from high school. Here, we fitted a scatter plot to a line.
# Logistic Regression is another form of regression. 
# However, Logistic Regression helps us determine if a point should be in category A or B, which is a perfect fit.
model = LogisticRegression(max_iter=1000) 
model.fit(X_train, y_train)


# Validate that the model has learned something.
# Recall the model operates on vectors. First transform the test set using the vectorizer. 
# Then generate the predictions.
y_pred = model.predict(X_test)


# We now want to see how we have done. We will be using three functions.
# `accuracy_score` tells us how well we have done. 
# 90% means that every 9 of 10 entries from the test dataset were predicted accurately.
# The `confusion_matrix` is a 2x2 matrix that gives us more insight.
# The top left shows us how many ham emails were predicted to be ham (that's good!).
# The bottom right shows us how many spam emails were predicted to be spam (that's good!).
# The other two quadrants tell us the misclassifications.
# Finally, the `classification_report` gives us detailed statistics which you may have seen in a statistics class.
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(classification_report(y_test, y_pred))



Accuracy: 97.58%
Confusion Matrix:
[[693  14]
 [ 11 314]]
Classification Report:
              precision    recall  f1-score   support

         ham       0.98      0.98      0.98       707
        spam       0.96      0.97      0.96       325

    accuracy                           0.98      1032
   macro avg       0.97      0.97      0.97      1032
weighted avg       0.98      0.98      0.98      1032



Step 4.

In [6]:
# Let's see which features (aka columns) the vectorizer created. 
# They should be all the words that were contained in the training dataset.
# Get the feature names (words) created by the vectorizer
feature_names = vecorizer.get_feature_names_out()
print("Features (words) created by the vectorizer:")
print(feature_names)



# You may be wondering what a machine learning model is tangibly. It is just a collection of numbers. 
# You can access these numbers known as "coefficients" from the coef_ property of the model
# We will be looking at coef_[0] which represents the importance of each feature.
# What does importance mean in this context?
# Some words are more important than others for the model.
# It's nothing personal, just that spam emails tend to contain some words more frequently.
# This indicates to the model that having that word would make a new email more likely to be spam.
# Get the coefficients from the model (importance of each feature)
coefficients = model.coef_[0]
feature_importance = list(zip(feature_names, coefficients))
feature_importance = sorted(feature_importance, key=lambda x: abs(x[1]), reverse=True)
print("Top 10 most important features (words) and their coefficients:")
for word, coef in feature_importance[:10]:
    print(f"{word}: {coef:.4f}")



# Iterate over importance and find the top 10 positive features with the largest magnitude.
# Similarly, find the top 10 negative features with the largest magnitude.
# Positive features correspond to spam. Negative features correspond to ham.
# You will see that `http` is the strongest feature that corresponds to spam emails. 
# It makes sense. Spam emails often want you to click on a link.
# Get the coefficients from the model (importance of each feature)
coefficients = model.coef_[0]
feature_importance = list(zip(feature_names, coefficients))

positive_features = [(word, coef) for word, coef in feature_importance if coef > 0]
negative_features = [(word, coef) for word, coef in feature_importance if coef < 0]

positive_features = sorted(positive_features, key=lambda x: abs(x[1]), reverse=True)
negative_features = sorted(negative_features, key=lambda x: abs(x[1]), reverse=True)

print("Top 10 positive features (indicative of spam):")
for word, coef in positive_features[:10]:
    print(f"{word}: {coef:.4f}")

print("\nTop 10 negative features (indicative of ham):")
for word, coef in negative_features[:10]:
    print(f"{word}: {coef:.4f}")




Features (words) created by the vectorizer:
['aa' 'aaa' 'aaas' ... 'zzo' 'zzocb' 'zzso']
Top 10 most important features (words) and their coefficients:
attached: -1.6914
enron: -1.5554
daren: -1.4290
thanks: -1.3323
doc: -1.2859
deal: -1.2375
meter: -1.1151
hpl: -1.1084
neon: -1.0008
xls: -0.9510
Top 10 positive features (indicative of spam):
http: 0.8944
hello: 0.8142
removed: 0.7805
prices: 0.7731
no: 0.7576
money: 0.7451
here: 0.7069
only: 0.6595
pain: 0.6476
more: 0.6354

Top 10 negative features (indicative of ham):
attached: -1.6914
enron: -1.5554
daren: -1.4290
thanks: -1.3323
doc: -1.2859
deal: -1.2375
meter: -1.1151
hpl: -1.1084
neon: -1.0008
xls: -0.9510


Submission
1. Upload the jupyter notebook to Forage.

All Done!